# LinkedIn Scraper
Este notebook coleta dados de perfis do LinkedIn que interagiram com postagens relacionadas à pesquisa "State of the Data 2024". O objetivo é extrair informações desses perfis e tentar reidentificá-los no dataset público divulgado pela pesquisa.


Coletamos dados de posts em que a pesquisa estava sendo divulgada para preenchimento. Dentre os posts coletados estão os seguintes:

https://www.linkedin.com/posts/crbazevedo_state-of-data-brazil-2024-activity-7272031218773225472-k2du/

https://www.linkedin.com/posts/paulovasconcellos_a-pesquisa-state-of-data-2425-já-está-no-activity-7252287169589878785-3tyi

https://www.linkedin.com/posts/data-hackers_data-dados-empresas-activity-6990305987286962176-O05v

Se fizer sentido coletar dados de mais pessoas, o seguinte post pode ser uma boa:

https://www.linkedin.com/posts/gabrielclages_um-pedido-rápido-e-direto-ao-ponto-se-você-activity-7262525821897113600-A7zt



## Imports

In [ ]:
from linkedin_scraper import actions
from linkedin_scraper import Person, Company
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from typing import Dict, List, Optional, Union
import time
import pandas as pd
import os
import re
import json
import openai
import pandas as pd
from bs4 import BeautifulSoup

## Autenticação e Navegação

Funções auxiliares e abertura da página. O login no linkedin deve ser feito manualmente.

In [ ]:
def setup_driver():
    return webdriver.Chrome()

def login(driver, email, password):
    actions.login(driver, email, password)

def open_page(driver, url):
    driver.get(url)
    time.sleep(3)

def save_to_csv(profiles, filename="linkedin_profiles.csv"):
    df = pd.DataFrame(profiles)
    file_exists = os.path.isfile(filename)

    df.to_csv(filename, mode='a', header=not file_exists, index=False, encoding="utf-8-sig")
    print(f"Adicionados {len(profiles)} perfis ao arquivo {filename}")

In [ ]:
driver = setup_driver()

In [ ]:
wait = WebDriverWait(driver, 10)

url = "https://www.linkedin.com/posts/gabrielclages_um-pedido-rápido-e-direto-ao-ponto-se-você-activity-7262525821897113600-A7zt"

open_page(driver, url)

## Coleta de perfis que interagiram
As duas funções abaixo fazem o seguinte:

 - Abrem o pupup com todas as pessoas que interagiram com o post
 - Rola a lista até o fim para carregar todas as pessoas
 - Coleta nome, link do perfil e descrição de cada um
 - Salva o resultado em 'linkedin_profiles.csv'

In [ ]:
def scroll_to_bottom_and_click_more(driver, wait, scrollable):
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable)

    while True:
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", scrollable)
        time.sleep(1)

        try:
            show_more = wait.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "button.scaffold-finite-scroll__load-button")
            ))
            # show_more.click()
            time.sleep(2)
        except:
            print("Botão 'Show more results' não encontrado ou não clicável.")
            break

        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable)
        if new_height == last_height:
            print("Nenhum conteúdo novo foi carregado.")
            break
        last_height = new_height

In [ ]:
def extract_profiles(driver):
    profiles = []
    profile_blocks = driver.find_elements(By.CSS_SELECTOR, "a.link-without-hover-state")

    for i, block in enumerate(profile_blocks):
        try:
            link = block.get_attribute("href")
            name_elem = block.find_element(By.CSS_SELECTOR, ".artdeco-entity-lockup__title span.text-view-model")
            name = name_elem.text.strip()

            desc_elem = block.find_element(By.CSS_SELECTOR, ".artdeco-entity-lockup__caption")
            description = desc_elem.text.strip()

            profiles.append({
                "Nome": name,
                "Link": link,
                "Descrição": description
            })
        except Exception as e:
            print(f"[{i}] Erro ao processar: {e}")
    
    print(f"Total de perfis extraídos: {len(profiles)}")
    
    return profiles

In [ ]:
elements = driver.find_elements(By.CSS_SELECTOR, ".social-details-social-counts__reactions-count")
for i, element in enumerate(elements):
    try:
        element.click()
        time.sleep(2)

        scrollable = driver.find_element(By.CSS_SELECTOR, "div.artdeco-modal__content")
        scroll_to_bottom_and_click_more(driver, wait, scrollable)
        profiles = extract_profiles(driver)
        save_to_csv(profiles)

        driver.find_element(By.CSS_SELECTOR, "button.artdeco-modal__dismiss").click()
        time.sleep(1)

    except Exception as e:
        print(f"Erro no elemento {i}: {e}")

Pode acontecer de uma mesma pessoa ter curtido dois posts que coletamos, então abaixo se extrai as entradas duplicadas.

In [ ]:
df = pd.read_csv('linkedin_profiles.csv')

df_unico = df.drop_duplicates(subset='Link')

df_unico.to_csv('linkedin_profiles.csv', index=False)

## Coleta de dados de cada um dos perfis

Optamos por acessar individualmente as páginas dos perfis coletados e salvar o HTML de cada uma delas na pasta /html_dumps, em vez de extrair diretamente as informações. Essa abordagem nos permite ajustar posteriormente o parsing dos dados, sem depender do acesso contínuo ao LinkedIn — o que reduz o risco de bloqueios.

Essa decisão se mostrou essencial, já que, após a visualização do centésimo perfil, o LinkedIn deixou de carregar as páginas normalmente. Para prosseguir além disso, seria necessário criar novas contas e/ou utilizar rotação de IP. Diante disso, optamos por restringir, ao menos inicialmente, a análise aos 100 primeiros perfis coletados.

In [ ]:
def save_linkedin_profile_html(driver, profile_url: str, output_dir: str = "html_dumps", timeout: int = 10) -> Optional[str]:

    try:
        os.makedirs(output_dir, exist_ok=True)

        driver.get(profile_url)

        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "main"))
        )

        html = driver.page_source

        profile_id = profile_url.rstrip('/').split('/')[-1]
        filename = f"{profile_id}.html"
        filepath = os.path.join(output_dir, filename)

        with open(filepath, "w", encoding="utf-8") as f:
            f.write(html)

        print(f"[✔] HTML salvo para {profile_url} em: {filepath}")
        return filepath

    except Exception as e:
        print(f"[✘] Falha ao salvar HTML de {profile_url}: {e}")
        return None

In [ ]:
df = pd.read_csv('linkedin_profiles.csv')

for index, row in df.iterrows():
    html_path = save_linkedin_profile_html(driver, row['Link'])

## Classificação e Análise

A partir desse ponto, o processo seguiu da seguinte forma: utilizamos três funções principais para extrair as informações relevantes de cada perfil. A primeira, `extrair_infos_gerais`, coleta dados como nome, cargo atual, localização e o link do perfil. Em seguida, a função `extrair_experiencias` reúne todas as experiências profissionais da pessoa, incluindo o cargo ocupado, o período de atuação e o link da empresa correspondente. Por fim, a função `extrair_educacao` obtém todas as formações acadêmicas listadas.

A partir das informações extraídas, estruturamos um dicionário por perfil e, em seguida, transformamos esses dados para o formato padronizado do dataset State of The Data. Os atributos extraídos, junto com o grau de confiança em cada um deles, são os seguintes:

* **UF onde a pessoa mora**: confiança **total**. Essa informação é declarada explicitamente no perfil, normalmente na seção de localização, e foi extraída de forma direta.

* **Tempo de experiência em dados**: confiança **moderada (\~70%)**. Dependemos das experiências listadas no perfil, que podem estar incompletas ou omitidas, além de possíveis diferenças na forma como o usuário entende e relata sua atuação na área.

* **Tempo prévio de experiência em TI**: confiança **moderada (\~70%)**, pelos mesmos motivos acima. Há chance de a pessoa não ter listado todo o histórico profissional ou de interpretar de forma distinta o que conta como experiência em TI.

* **Nível mais alto de escolaridade até novembro de 2024**: confiança **quase total**. É altamente provável que o LinkedIn reflita corretamente a formação mais avançada do usuário, e conseguimos capturar essa informação com precisão.

* **Área principal de formação**: confiança **quase total**. A área de formação costuma estar bem especificada no perfil, e foi extraída diretamente da seção de educação.

* **Cargo atual em novembro de 2024**: confiança **quase total**. Utilizamos o cargo listado mais recentemente até a data de corte e o normalizamos conforme as categorias do dataset, o que nos dá alta confiança na correspondência.

* **Link da empresa em novembro de 2024**: **dado auxiliar, ainda não um atributo final**. Embora tenhamos coletado o link da empresa referente ao cargo atual, ele será utilizado posteriormente para enriquecer os dados com o **setor da empresa** e o **número de funcionários**, atributos que ainda serão incluídos.

Essas informações foram passadas para uma LLM, junto com os possíveis valores esperados para cada campo, a fim de realizar a classificação e gerar uma versão estruturada para cada perfil. Por fim, salvamos o resultado em um arquivo CSV chamado `linkedin_profiles_data.csv`.


In [ ]:
openai.api_key = ""
DIRETORIO_HTML = "html_dumps"
ARQUIVO_SAIDA = "linkedin_profiles_data.csv"

def limpar_json(raw):
    if raw.startswith("```"):
        raw = raw.strip().strip("`")
        inicio = raw.find("{")
        fim = raw.rfind("}")
        if inicio != -1 and fim != -1:
            raw = raw[inicio:fim+1]
    return raw

In [ ]:
def extrair_infos_gerais(soup):
    nome_tag = soup.find("h1", class_="aYftUZQtwiflyYvLNDOYHdctvSKFncppEBZDg inline t-24 v-align-middle break-words")
    nome = nome_tag.get_text(strip=True) if nome_tag else "N/A"

    cargo_tag = soup.find("div", class_="text-body-medium break-words")
    cargo = cargo_tag.get_text(strip=True) if cargo_tag else "N/A"

    local_tag = soup.find("span", class_="text-body-small inline t-black--light break-words")
    localizacao = local_tag.get_text(strip=True) if local_tag else "N/A"

    link_tag = soup.find("a", class_="IqLKNJXIVaTxfTzhuwyMDMsOBGXIClUI")
    href = link_tag["href"] if link_tag else None
    if href and href.startswith("/in/"):
        base_href = href.split("/overlay")[0]
        link_perfil = f"https://www.linkedin.com{base_href}/"
    else:
        link_perfil = "N/A"

    return nome, cargo, localizacao, link_perfil

In [ ]:
def encontrar_secao_experiencia(soup):
    for section in soup.find_all("section"):
        h2 = section.find("h2", class_="pvs-header__title")
        if h2 and "Experiência" in h2.get_text():
            return section
    return None

def extrair_experiencias(section):
    experiencias = []
    blocos_empresa = section.find_all("div", attrs={"data-view-name": "profile-component-entity"}, recursive=True)

    for bloco in blocos_empresa:
        is_empresa = bloco.find("div", class_=lambda c: c and "flex-column" in c)
        if not is_empresa:
            continue

        link_empresa_tag = bloco.find("a", href=True)
        link_empresa = f"https://www.linkedin.com{link_empresa_tag['href']}" if link_empresa_tag else "N/A"

        subcargos = bloco.find_all("div", attrs={"data-view-name": "profile-component-entity"}, recursive=False)
        if len(subcargos) > 1:
            for sub in subcargos:
                cargo = sub.find("div", class_=lambda c: c and "t-bold" in c)
                cargo_span = cargo.find("span", attrs={"aria-hidden": "true"}) if cargo else None
                nome_cargo = cargo_span.get_text(strip=True) if cargo_span else "N/A"

                periodo_tag = sub.find("span", class_="pvs-entity__caption-wrapper")
                periodo = periodo_tag.get_text(strip=True) if periodo_tag else "N/A"

                experiencias.append({
                    "cargo": nome_cargo,
                    "periodo": periodo,
                    "link_empresa": link_empresa
                })
        else:
            cargo_tag = bloco.find("div", class_=lambda c: c and "t-bold" in c)
            cargo_span = cargo_tag.find("span", attrs={"aria-hidden": "true"}) if cargo_tag else None
            nome_cargo = cargo_span.get_text(strip=True) if cargo_span else "N/A"

            periodo_tag = bloco.find("span", class_="pvs-entity__caption-wrapper")
            periodo = periodo_tag.get_text(strip=True) if periodo_tag else "N/A"

            experiencias.append({
                "cargo": nome_cargo,
                "periodo": periodo,
                "link_empresa": link_empresa
            })

    return experiencias

In [ ]:
def extrair_educacao(soup):
    educacoes = []
    for section in soup.find_all("section"):
        h2 = section.find("h2", class_="pvs-header__title")
        if h2 and "Formação acadêmica" in h2.get_text(strip=True):
            ul = section.find("ul")
            if not ul:
                continue
            for li in ul.find_all("li", recursive=False):
                instituicao_span = li.find("div", class_="t-bold")
                if instituicao_span:
                    instituicao_span = instituicao_span.find("span", attrs={"aria-hidden": "true"})
                curso_span = li.find("span", class_="t-14 t-normal")
                if curso_span:
                    curso_span = curso_span.find("span", attrs={"aria-hidden": "true"})
                periodo_span = li.find("span", class_="t-14 t-normal t-black--light")
                if periodo_span:
                    periodo_span = periodo_span.find("span", attrs={"aria-hidden": "true"})

                educacoes.append({
                    "instituicao": instituicao_span.get_text(strip=True) if instituicao_span else None,
                    "curso": curso_span.get_text(strip=True) if curso_span else None,
                    "periodo": periodo_span.get_text(strip=True) if periodo_span else None
                })
            break
    return educacoes

In [ ]:
def classificar_perfil_com_llm(dados_perfil: dict) -> dict:
    prompt = f"""
A seguir está um dicionário com informações extraídas de um perfil do LinkedIn. Considere **novembro de 2024** como a data de referência para responder às perguntas abaixo. Algumas experiências podem conter ruído (por exemplo, apenas o nome da empresa ou informações incompletas), então desconsidere qualquer entrada que não pareça de fato um cargo exercido.

Classifique com base nas seguintes opções:

1. **UF onde mora** (`uf_onde_mora`): sigla do estado.
['RS', 'SC', 'SP', 'DF', 'MA', 'BA', 'MG', 'PR', 'MT', 'GO', 'AL', 'PB', 'PE', 'RJ', 'ES', 'AP', 'CE', 'TO', 'PI', 'MS', 'RN', 'AM', 'RO', 'SE', 'PA']

2. **Tempo de experiência na área de dados** (`tempo_de_experiencia_em_dados`):
['Menos de 1 ano', 'de 1 a 2 anos', 'de 3 a 4 anos', 'de 5 a 6 anos', 'de 7 a 10 anos', 'Mais de 10 anos', 'Não tenho experiência na área de dados']

3. **Tempo de experiência em TI antes de atuar com dados** (`tempo_de_experiencia_em_ti`):
['Menos de 1 ano', 'de 1 a 2 anos', 'de 3 a 4 anos', 'de 5 a 6 anos', 'de 7 a 10 anos', 'Mais de 10 anos', 'Não tive experiência na área de TI/Engenharia de Software antes de começar a trabalhar na área de dados']

4. **Nível de ensino mais alto alcançado até novembro de 2024** (`nivel_de_ensino`):
['Estudante de Graduação', 'Graduação/Bacharelado', 'Pós-graduação', 'Mestrado', 'Doutorado ou Phd', 'Não tenho graduação formal', 'Prefiro não informar']

5. **Área de formação principal** (`area_de_formação`):
['Computação / Engenharia de Software / Sistemas de Informação/ TI', 'Economia/ Administração / Contabilidade / Finanças/ Negócios', 'Estatística/ Matemática / Matemática Computacional/ Ciências Atuariais', 'Outra opção', 'Outras Engenharias (não incluir engenharia de software ou TI)', 'Ciências Biológicas/ Farmácia/ Medicina/ Área da Saúde', 'Marketing / Publicidade / Comunicação / Jornalismo / Ciências Sociais', 'Química / Física']

7. **Cargo atual da pessoa em novembro de 2024** (`cargo_atual`):
['Analista de Dados/Data Analyst', 'Analista de BI/BI Analyst', 'Cientista de Dados/Data Scientist', 'Engenheiro de Dados/Data Engineer/Data Architect', 'Engenheiro de Machine Learning/ML Engineer/AI Engineer', 'Analytics Engineer', 'Data Product Manager/ Product Manager (PM/APM/DPM/GPM/PO)', 'Analista de Negócios/Business Analyst', 'Analista de Suporte/Analista Técnico', 'Professor/Pesquisador', 'Desenvolvedor/ Engenheiro de Software/ Analista de Sistemas', 'Arquiteto de Dados/Data Architect', 'Estatístico', 'Outra Opção', 'Outras Engenharias (não inclui dev)']

8. **Link da empresa onde a pessoa estava em novembro de 2024** (`link_da_empresa_em_novembro_2024`): se conseguir identificar

Retorne no seguinte formato JSON:
{{
    "uf_onde_mora": "",
    "tempo_de_experiencia_em_dados": "",
    "tempo_de_experiencia_em_ti": "",
    "nivel_de_ensino": "",
    "area_de_formação": "",
    "cargo_atual": "",
    "link_da_empresa_em_novembro_2024": ""
}}

Dicionário com dados do perfil:
{json.dumps(dados_perfil, ensure_ascii=False, indent=2)}
""".strip()

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        temperature=0,
        messages=[
            {"role": "system", "content": "Você é um classificador de perfis de LinkedIn."},
            {"role": "user", "content": prompt}
        ]
    )

    resposta_texto = response.choices[0].message.content.strip()
    try:
        resposta_limpa = limpar_json(resposta_texto)
        return json.loads(resposta_limpa)
    except Exception as e:
        print("Erro ao interpretar resposta da LLM:", e)
        print("Resposta bruta:", resposta_texto)
        return {}

In [ ]:
data = []

# for filename in os.listdir(DIRETORIO_HTML):
#     if not filename.endswith(".html"):
#         continue

filepath = os.path.join(DIRETORIO_HTML, 'ACoAAAH4yPQBCgngA7AVez1BQFxvXL67TcUwj4E.html')
with open(filepath, "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

nome, cargo, localizacao, link_perfil = extrair_infos_gerais(soup)
secao_experiencia = encontrar_secao_experiencia(soup)
experiencias = extrair_experiencias(secao_experiencia) if secao_experiencia else []
educacoes = extrair_educacao(soup)

dados_perfil = {
    "nome": nome,
    "cargo_atual": cargo,
    "localizacao": localizacao,
    "experiencias": experiencias,
    "formacoes": educacoes,
    "link_perfil": link_perfil
}

resumo = classificar_perfil_com_llm(dados_perfil)
data.append({
    "nome": nome,
    "localizacao_bruta": localizacao,
    "link_perfil": link_perfil,
    "cargo_bruto": cargo,
    "uf_onde_mora": resumo.get("uf_onde_mora", ""),
    "tempo_de_experiencia_em_dados": resumo.get("tempo_de_experiencia_em_dados", ""),
    "tempo_de_experiencia_em_ti": resumo.get("tempo_de_experiencia_em_ti", ""),
    "nivel_de_ensino": resumo.get("nivel_de_ensino", ""),
    "area_de_formação": resumo.get("area_de_formação", ""),
    "cargo_atual": resumo.get("cargo_atual", ""),
    "link_da_empresa_em_novembro_2024": resumo.get("link_da_empresa_em_novembro_2024", "")
})

print({
    "nome": nome,
    "localizacao_bruta": localizacao,
    "link_perfil": link_perfil,
    "cargo_bruto": cargo,
    "uf_onde_mora": resumo.get("uf_onde_mora", ""),
    "tempo_de_experiencia_em_dados": resumo.get("tempo_de_experiencia_em_dados", ""),
    "tempo_de_experiencia_em_ti": resumo.get("tempo_de_experiencia_em_ti", ""),
    "nivel_de_ensino": resumo.get("nivel_de_ensino", ""),
    "area_de_formação": resumo.get("area_de_formação", ""),
    "cargo_atual": resumo.get("cargo_atual", ""),
    "link_da_empresa_em_novembro_2024": resumo.get("link_da_empresa_em_novembro_2024", "")
})

df = pd.DataFrame(data)
df.to_csv(ARQUIVO_SAIDA, index=False, encoding="utf-8")
print(f"✅ Arquivo '{ARQUIVO_SAIDA}' criado com sucesso.")

### Próximos Passos — Enriquecimento com dados das empresas

Como sugestão de melhoria, o próximo passo seria enriquecer o dataset `linkedin_profiles_data.csv` com informações das empresas onde os profissionais estavam atuando em **novembro de 2024**.

A última coluna do dataset já contém o **link para o perfil da empresa** no LinkedIn. A partir desses links, o ideal fazer um parser para acessar a página de cada empresa e coletar:

* **Setor da empresa** (ex: Fintech, Educação, Saúde, etc.)
* **Número de funcionários** (em faixas como "11–50", "201–500", etc.)

**Observação importante**:

O LinkedIn geralmente **não exibe o setor da empresa de forma estruturada**. No entanto, a **seção "Sobre" (About)** costuma conter uma descrição textual rica que pode ser utilizada como fonte. A ideia seria:

1. Extrair o texto da seção "Sobre" de cada empresa.
2. Passar esse texto para uma **LLM**, junto com os **valores possíveis de setores** (obtidos a partir dos `unique()` do dataset original), os quais são os seguintes: 

['Marketing' 'Finanças ou Bancos' 'Indústria'
 'Tecnologia/Fábrica de Software' nan 'Telecomunicação' 'Outra Opção'
 'Setor Público' 'Área da Saúde' 'Área de Consultoria' 'Varejo'
 'Entretenimento ou Esportes' 'Setor Automotivo'
 'Setor Imobiliário/ Construção Civil' 'Educação' 'Internet/Ecommerce'
 "Filantropia/ONG's" 'Seguros ou Previdência' 'Agronegócios'
 'Setor de Energia' 'Setor Alimentício' 'Setor Farmaceutico']

3. Solicitar à LLM que classifique a empresa no setor mais adequado com base no conteúdo do texto.

O mesmo pode ser feito para o **número de funcionários**. Embora o Linkedin exiba essa informação, o **formato pode variar** e pode não estar padronizado conforme as categorias do nosso dataset. Por isso, a ideia também é:

1. Extrair a estimativa bruta do número de funcionários (ex: "201-500 employees").
2. Passar esse valor para a **LLM**, junto com as faixas esperadas no dataset, as quais são:

['de 101 a 500' 'Acima de 3.000' 'de 501 a 1.000' 'de 1.001 a 3.000' nan
 'de 6 a 10' 'de 11 a 50' 'de 1 a 5' 'de 51 a 100']